In [0]:
from pyspark.sql.functions import col
file_location = "/Workspace/Users/ts149509@student.sgh.waw.pl/Data/telco.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

from pyspark.sql.functions import trim, when 

#df_tc_dbl = df.withColumn('TotalCharges', col('TotalCharges').cast('double'))

#display(df.select(trim(col("TotalCharges"))))
#.cast('double')
df_tc_dbl = df.withColumn('TotalCharges', trim(col("TotalCharges")))

#df_tc_dbl['TotalCharges Trimmed'] = df_tc_dbl['TotalCharges Trimmed'].apply(lambda x: float(x) if x != '' else 0)
#df_tc_dbl.replace()
#df_tc_dbl['TotalCharges Trimmed'] = df_tc_dbl['TotalCharges Trimmed'].replace('', 0)

df_tc_dbl = df.withColumn("TotalCharges", \
       when(col('TotalCharges')==" " , None) \
          .otherwise(col('TotalCharges')))
df_tc_dbl = df_tc_dbl.withColumn('TotalCharges', col('TotalCharges').cast('double'))

#df_tc_dbl = df.withColumn('TotalCharges', df.select(trim(col("TotalCharges"))).cast('double'))
display(df_tc_dbl)

In [0]:
df_grp_contract_phone = df_tc_dbl.groupBy('Contract', 'PhoneService').count().sort(['Contract', 'PhoneService'])
#df_grp_contract_phone.show()
display(df_grp_contract_phone)

In [0]:
from pyspark.sql.functions import round, avg, count
# Aggregate: Contract, PhoneService, Avg MonthlyCharges (rounded), count (rows)
df_agg = df_tc_dbl.groupBy('Contract', 'PhoneService') \
    .agg(round(avg('MonthlyCharges'), 2).alias('Avg MonthlyCharges'), count('Contract').alias('Count'))
display(df_agg)


In [0]:
type(df_agg.collect())
#df_agg.collect()
my_res = None
for row in df_agg.collect():
  #print(row)
  row_dict = row.asDict()
  #print(type(row_dict))
  if row_dict['Contract'] == 'One year' and row_dict['PhoneService'] == 'Yes':
    my_res = row_dict['Avg MonthlyCharges']
    print(type(my_res))
    print(my_res)

In [0]:
df_selection = df_tc_dbl.select('Contract', 'PhoneService', 'MonthlyCharges')
df_filtered = df_selection.filter((df_selection['Contract'] == 'One year') | (df_selection['PhoneService'] == 'Yes'))  # OR

df_filtered = df_selection.filter((df_selection['Contract'] == 'One year') & (df_selection['PhoneService'] == 'Yes'))  # AND

display(df_filtered)

In [0]:

# Find rows for which TotalCharges are not equal to MonthlyCharges * tenure

# ===

df_res = df_tc_dbl.withColumn('Tenure x Monthly Charges', round(col('tenure') * col('MonthlyCharges'), 2))
df_res = df_res.withColumn('Diff Charges', round(col('Tenure x Monthly Charges') - col('TotalCharges'), 2))
#display(df_res)
eq = df_res.filter(col('Diff Charges') == 0).count()
print(f'Equal: {eq}')
df_res = df_res.filter(col('Diff Charges') != 0)
neq = df_res.count()
print(f'Not Equal: {neq}')
#display(df_res)
from pyspark.sql.functions import desc
df_res2 = df_res.sort(desc('Diff Charges')).limit(10)
display(df_res2)


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def convert_contract(cntr):
    cntr = cntr.replace('Month-to-month', 'monthly')
    cntr = cntr.replace('One', '1')
    cntr = cntr.replace('Two', '2')
    return cntr

df_res3 = df_res.withColumn('Contract Short', convert_contract(col('Contract')))
display(df_res3)

TODO: Check if customers with dependents pay on average more than those without dependents. The answer should be a single bool (True or False)

In [0]:
# Check if customers with dependents pay on average more than those without dependents. The answer should be a single bool (True or False)

from pyspark.sql.functions import avg, mean, col

df_res3.groupBy('Dependents').agg(mean('MonthlyCharges').alias('Avg Monthly Charge')).display()


